# 어떤 단어가 많이 쓰였는가? 

In [1]:
from konlpy.tag import Okt
import re
import pandas as pd
import numpy as np

# Bag of Words(BOW)란?
1. 정의 : 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도에만 집중하는 텍스트 데이터의 수치화 표현 방법


2. BoW를 만드는 과정

 (1) 우선, 각 단어에 고유한 정수 인덱스를 부여합니다.
 
 (2) 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만듭니다.

### 예제1
- 문서1 : 정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.

In [3]:
okt=Okt()
#정규 표현식을 통해 온점을 제거하는 정제 작업입니다.
token=re.sub("(\.)","",
             "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.")

In [4]:
#OKT 형태소 분석기를 통해 토큰화 작업을 수행한 뒤에, token에다가 넣습니다.
token=okt.morphs(token)

In [6]:
word2index={}
bow=[]

for voca in token:
    if voca not in word2index.keys():
        word2index[voca]=len(word2index)
# token을 읽으면서, word2index에 없는 (not in) 단어는 새로 추가하고, 이미 있는 단어는 넘깁니다.
        bow.insert(len(word2index)-1,1)
# BoW 전체에 전부 기본값 1을 넣어줍니다. 단어의 개수는 최소 1개 이상이기 때문입니다.
    else:
        index=word2index.get(voca)
        # 재등장하는 단어의 인덱스를 받아옵니다.
        bow[index]=bow[index]+1
        # 재등장한 단어는 해당하는 인덱스의 위치에 1을 더해줍니다.
        # 단어의 개수를 세는 것입니다.
print(word2index)

{'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}


In [7]:
bow

[1, 2, 1, 1, 2, 1, 1, 1, 1, 1]

### 예제2
- 문서2 : 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.

In [8]:
token2=re.sub("(\.)","",
             "소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.")
token2=okt.morphs(token2)
word2index2={}
bow2=[]

for voca in token2:
    if voca not in word2index2.keys():
        word2index2[voca]=len(word2index2)
        bow2.insert(len(word2index2)-1,1)
    else:
        index=word2index2.get(voca)
        bow2[index]=bow2[index]+1
print(word2index2)

{'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}


In [9]:
bow2

[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]

### 예제3
- 문서3 : 정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다. 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.

In [10]:
token3=re.sub("(\.)","",
             "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다. 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.")
token3=okt.morphs(token3)
word2index3={}
bow3=[]

for voca in token3:
    if voca not in word2index3.keys():
        word2index3[voca]=len(word2index3)
        bow3.insert(len(word2index3)-1,1)
    else:
        index=word2index3.get(voca)
        bow3[index]=bow3[index]+1
print(word2index3)

{'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17}


In [11]:
bow3

[1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]

# CountVectorizer 클래스로 BoW 만들기
- 사이킷 런에서 단어의 빈도를 Count하여 Vector로 만드는 CountVectorizer 클래스 지원

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
corpus=['you know I want your love. because I love you.']
vector=CountVectorizer()
print(vector.fit_transform(corpus).toarray())
print(vector.vocabulary_)

[[1 1 2 1 2 1]]
{'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


 - CountVectorizer : 기본적으로 길이가 2이상인 문자에 대해서만 토큰으로 인식
 - CountVectorizer 주의점 : 단지 띄어쓰기만을 기준으로 단어를 자르는 낮은 수준의 토큰화를 진행함

### scikit-learn 문서 전처리 기능
1. DictVectorizer : 각 단어의 수를 세어놓은 사전에서 BoW 인코딩 벡터를 만듦.
2. CountVectorizer : 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BoW 인코딩 벡터를 만듦.
3. TfidVectorizer : CountVectorizer와 비슷하지만 TF-IDF 방식으로 단어의 가중치를 조정한 BoW 인코딩 벡터를 만듦.
4. HashingVectorizer : 해시 함수를 사용하여 적은 메모리와 빠른 속도로 BoW 인코딩 벡터를 만듦.

### CountVectorizer는 이러한 작업을 하기 위한 다음과 같은 인수를 가질 수 있다.
 - stop_words : 문자열 {‘english’}, 리스트 또는 None (디폴트)
    - stop words 목록.‘english’이면 영어용 스탑 워드 사용.
    - 문서에서 단어장을 생성할 때 무시할 수 있는 단어를 말한다. 보통 영어의 관사나 접속사, 한국어의 조사 등이 여기에 해당
 - analyzer : 문자열 {‘word’, ‘char’, ‘char_wb’} 또는 함수
    - 단어 n-그램, 문자 n-그램, 단어 내의 문자 n-그램
 - token_pattern : string
    - 토큰 정의용 정규 표현식
 - tokenizer : 함수 또는 None (디폴트)
    - 토큰 생성 함수 .
 - ngram_range : (min_n, max_n) 튜플
    - n-그램 범위
 - max_df : 정수 또는 [0.0, 1.0] 사이의 실수. 디폴트 1
    - 단어장에 포함되기 위한 최대 빈도
 - min_df : 정수 또는 [0.0, 1.0] 사이의 실수. 디폴트 1
    - 단어장에 포함되기 위한 최소 빈도

# 불용어를 제거한 BoW 만들기
- 불용어 :  자연어 처리에서 별로 의미를 갖지 않는 단어들

### (1) 사용자가 직접 정의한 불용어 사용

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
text=["Family is not an important thing. It's everything."]
vect=CountVectorizer(stop_words=['the','a','an','is','not'])
print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


### (2) CounterVectorizer에서 제공하는 자체 불용어 사용

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
text=["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words="english") #한글은 없는 것 같다
print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1]]
{'family': 0, 'important': 1, 'thing': 2}


### NLTK에서 지원하는 불용어 사용

In [16]:
#에러 떠서 이걸 깔아야 합니다.
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
text=["Family is not an important thing. It's everything."]
from nltk.corpus import stopwords
sw = stopwords.words("english")
vect = CountVectorizer(stop_words =sw)
print(vect.fit_transform(text).toarray()) 
print(vect.vocabulary_)

[[1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


# 문서 단어 행렬(Document-Term Matrix,DTM) 
- 각 문서에 대한 BoW 표현 방법을 그대로 갖고 와서, 서로 다른 문서들의 BoW들을 결합한 표현 방법

## 1. 문서 단어 행렬의 표기법
- 문서 단어 행렬 : 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것
- 각 문서에 대한 BoW를 하나의 행룔로 만든 것
- BoW와 다른 표현 방법이 아니라 BoW 표현을 다수의 문서에 대해서 행렬로 표현하고 부르는 용어
- 의의 : 문서들을 서로 비교할 수 있또록 수치화할 수 있다는 점

## 2. 문서 단어 행렬의 한계
### 1) 희소 표현(Sparse representataion)
- 희소 벡터는 많은 양의 저장 공간과 계산을 위한 리소스를 필요
- 각 문서 벡터의 차원은 원-핫 벡터와 마찬가지로 전체 단어 집합의 크기를 가짐
- 전처리를 통해 단어 집합의 크기를 줄이는 일은 BoW 표현을 사용하는 모델에서 중요
- 구두점, 빈도수가 낮은 단어, 불용어를 제거하고, 어간이나 표제어 추출을 통해 단어를 정규화하여 단어 집합의 크기를 줄일 수 있음

### 2) 단순 빈도 수 기반 접근
- 불용어 때문에 유사한 문서 비교하기 어려울 수 있음
- 불용어를 빼는 방법을 통해 해당 문제 제거할 수 있음
- 불용어와 중요한 단어에 대해서 가중치를 줄 수 있는 TF-IDF를 사용하면 해결할 수 있음

# TF-IDF(Term Frequency-Inverse Document Frequency)
- 단어의 빈도와 역 문서 빈도(문서의 빈도에 특정 식을 취함)를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법
- 사용 방법 : DTM을 만든 후, TF-IDF 가중치를 부여
- 용도 : 주로 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 쓰일 수 있음
- TF x IDF = 문서(d),단어(t),문서의 총 개수(n)

### (1) tf(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수
- TF : DTM의 각 단어들이 가진 값들
    - DTM이 각 문서에서의 각 단어의 등장 빈도를 나타내는 값

### (2) df(t) : 특정 단어 t가 등장한 문서의 수
- 오직 특정 단어 t가 등장한 문서의 수에만 관심을 가짐

### (3) idf(d,t) : df(t)에 반비례하는 수
- idf(d,t)=log(n/(1+df(t)))
- DF의 역수

- TF-IDF는 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단하며 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단함.
- TF-IDF 값이 낮으면 중요도가 낮은 것, TF-IDF 값이 크면 중요도가 큰 것

## 사이킷런을 이용한 DTM과 TF-IDF 실습

### DTM은 BoW를 만들 때 사용한 CountVectorizer를 사용하면 간단히 만들 수 있음

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도 수를 기록한다.
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


### 사이킷런은 TF-IDF를 자동 계산해주는 TfidVectorizer 제공
- 사이킷런의 TF-IDF는 위에서 배웠던 보편적인 TF-IDF 식에서 좀 더 조정된 다른 식 사용
- IDF 계산 시 분자에다가도 1을 더해주며, TF-IDF에 L2 정규화라는 방법으로 값을 조정함

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


# 케라스로 DTM과 TF-IDF 행렬 만들기
- 다층 퍼셉트론(MLP)

## 1. 케라스의 texts_to_matrix() 이해하기

In [22]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

In [23]:
texts=['먹고 싶은 사과', '먹고 싶은 바나나', 
       '길고 노란 바나나 바나나', '저는 과일이 좋아요']

In [24]:
t = Tokenizer()
t.fit_on_texts(texts)
print(t.word_index)

{'바나나': 1, '먹고': 2, '싶은': 3, '사과': 4, '길고': 5, '노란': 6, '저는': 7, '과일이': 8, '좋아요': 9}


In [25]:
print(t.texts_to_matrix(texts, mode='count'))

[[0. 0. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]]


- texts_to_matrx()는 총 4개의 모드를 지원하는데 각 모드는 'binary', 'count', 'freq', 'tfidf'로 총 4개
- count : DTM 생성
- binary : 해당 단어가 존재하는지만 관심을 가지고 해당 단어가 몇 개 였는지는 무시
- tfidf : TF-IDF 행렬을 만듦.
- freq : 각 문서에서의 각 단어의 등장 횟수를 분자, 각 문서의 크기(각 문서에서 등장한 모든 단어의 개수의 총합)를 분모로 하는 표현 방법

## 코랩 버전으로 MLP 돌리기